# Traitement de données graphes avec Spark GraphX

Dans ce notebook, nous allons utiliser Spark GraphX pour récupérer des information sur le graphe suivant, représentant un réseau social fictif : 

![Graphe](../images/graph.png)

# Démarrage du notebook et importation des librairies nécessaires

In [1]:
print("Hello, World from Scala kernel!")

Intitializing Scala interpreter ...

Spark Web UI available at http://3c99054b149a:4042
SparkContext available as 'sc' (version = 3.5.0, master = local[*], app id = local-1704581855162)
SparkSession available as 'spark'


Hello, World from Scala kernel!

In [2]:
import org.apache.spark._;
import org.apache.spark.rdd.RDD;
import org.apache.spark.util.IntParam;
import org.apache.spark.graphx._;
import org.apache.spark.graphx.util.GraphGenerators;

import org.apache.spark._
import org.apache.spark.rdd.RDD
import org.apache.spark.util.IntParam
import org.apache.spark.graphx._
import org.apache.spark.graphx.util.GraphGenerators


# Création du graphe

On créé d'abord les noeuds et les arêtes, que l'on transforme ensuite en RDDs pour qu'ils soient utilisables par Spark GraphX :

In [4]:
val nodes = sc.parallelize(
    Array(
        (1L, ("Alia", 28)), (2L, ("Bechir", 27)), (3L, ("Chiheb", 65)),
        (4L, ("Delia", 42)), (5L, ("Emir", 35)), (6L, ("Fawzi", 50))
    )
)
val edges = sc.parallelize(
    Array(
        Edge(2,1,"follows"), Edge(4,1,"follows"), Edge(2,4,"follows"), Edge(3,2,"follows"),
        Edge(5,2,"follows"), Edge(5,3,"follows"), Edge(3,6,"follows"), Edge(6,5,"follows")
    )
)

nodes: org.apache.spark.rdd.RDD[(Long, (String, Int))] = ParallelCollectionRDD[0] at parallelize at <console>:33
edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[String]] = ParallelCollectionRDD[1] at parallelize at <console>:40


On peut ensuite créer le graphe à partir des RDDs :

In [5]:
val graph = Graph(nodes, edges)

graph: org.apache.spark.graphx.Graph[(String, Int),String] = org.apache.spark.graphx.impl.GraphImpl@742bf6bc


# Exploration des données : exercices

Question 1 : Quel est le nombre d'utilisateurs du réseau social ?

In [51]:
println("Il y a " + graph.numVertices + " utilisateurs dans ce réseau social.")

Il y a 6 utilisateurs dans ce réseau social.


Question 2 : Afficher tous les utlisateurs du réseau social (nom et âge).

In [56]:
graph.vertices.collect.foreach { case (id, (name, age)) => println(s"$name : $age ans")}

Alia : 28 ans
Bechir : 27 ans
Chiheb : 65 ans
Delia : 42 ans
Emir : 35 ans
Fawzi : 50 ans


Question 3 : Quel est le nombre total d'abonnés pour chaque utilisateur?

In [83]:
// Quel est le nombre total d'abonnés pour chaque utilisateur? (afficher le nom de l'utilisateur et le nombre d'abonnés)

val numberOfFollowersPerUser = graph.inDegrees

graph.vertices.map { case (id, (name, age)) => (id, name) }.join(numberOfFollowersPerUser).map { case (id, (name, count)) => (name, count) }.collect.foreach { case (name, count) => println(s"$name a $count abonnés") }

Alia a 2 abonnés
Bechir a 2 abonnés
Chiheb a 1 abonnés
Delia a 1 abonnés
Emir a 1 abonnés
Fawzi a 1 abonnés


numberOfFollowersPerUser: org.apache.spark.graphx.VertexRDD[Int] = VertexRDDImpl[111] at RDD at VertexRDD.scala:57


Question 4 : Afficher tous les "follows" du réseau social

In [70]:
val follows = graph.edges.filter { case Edge(src, dst, prop) => prop == "follows" }.collect()

follows.foreach { case Edge(src, dst, prop) => println(s"${graph.vertices.filter { case (id, (name, age)) => id == src }.first._2._1} follows ${graph.vertices.filter { case (id, (name, age)) => id == dst }.first._2._1}") }

Bechir follows Alia
Delia follows Alia
Bechir follows Delia
Chiheb follows Bechir
Emir follows Bechir
Emir follows Chiheb
Chiheb follows Fawzi
Fawzi follows Emir


follows: Array[org.apache.spark.graphx.Edge[String]] = Array(Edge(2,1,follows), Edge(4,1,follows), Edge(2,4,follows), Edge(3,2,follows), Edge(5,2,follows), Edge(5,3,follows), Edge(3,6,follows), Edge(6,5,follows))


Question 5 : Quels sont les utilisateurs qui suivent Alia (id 1) ?

In [73]:
val aliaFollowersEdges = graph.edges.filter { case Edge(src, dst, prop) => dst == 1 }.map { case Edge(src, dst, prop) => src }.collect()
val aliaFollowers = graph.vertices.filter { case (id, (name, age)) => aliaFollowersEdges.contains(id) }.map { case (id, (name, age)) => name }.collect()

println("Les utilisateurs qui suivent Alia sont " + aliaFollowers.mkString(", "))

Les utilisateurs qui suivent Alia sont Bechir, Delia


aliaFollowersEdges: Array[org.apache.spark.graphx.VertexId] = Array(2, 4)
aliaFollowers: Array[String] = Array(Bechir, Delia)
